In [ ]:
import pandas as pd
import geopandas as gpd
import requests
import json
import pandas as pd
import numpy as np
import time
import os

try :
    import xmltodict
except :
    !pip install xmltodict
    import xmltodict

### 1. 지오코딩

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/공모전/천안시/지오코딩 데이터_천안.xlsx")
df.head()

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/공모전/천안시/지오코딩 데이터_천안.xlsx")
df.head()

In [ ]:
df_1 = df.drop_duplicates('지번주소', keep='first').reset_index(drop=True)
df_1

In [ ]:
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup

In [ ]:
client_id = 'm5pfmpajew'                                    ##### 제 개인 client_ID, PW입니당...^^... 사용하실 땐 ID/PW를 바꿔주십쇼 #####
client_pw = 'tRnwWCDO9RrCMdnFHg8MrVFpCKOqrwsuu19YaVp2'      ##### 제 개인 client_ID, PW입니당...^^... 사용하실 땐 ID/PW를 바꿔주십쇼 #####

api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

In [ ]:
geo_coordi_list = []
road_add_list = []
jibun_add_list = []
address_ele_lit = []
i = 0
start_time = time.time()

for add in df_1['지번주소']:
    add_urlenc = parse.quote(add)
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)

    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode() # 정상이면 200 리턴
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)
            if 'addresses' in response_body:
                try:
                    road_address = response_body['addresses'][0]['roadAddress']
                    jibun_address = response_body['addresses'][0]['jibunAddress']
                    #address_elements = response_body['addresses'][0]['addressElements']
                    latitude = response_body['addresses'][0]['y']
                    longitude = response_body['addresses'][0]['x']
                    if i%100 == 0:
                        print('100 results Success!')
                except:
                    road_address = None
                    jibun_address = None
                    #address_elements = None
                    latitude = None
                    longitude = None
            else:
                print('resulut not exist!')
                road_address = None
                jibun_address = None
                #address_elements = None
                latitude = None
                longitude = None
        else:
            print(f'Response error code: {rescode}')
            road_address = None
            jibun_address = None
            #address_elements = None
            latitude = None
            longitude = None

    geo_coordi_list.append([latitude, longitude])
    road_add_list.append(road_address)
    jibun_add_list.append(jibun_address)
    i += 1

print(f"총 소요시간: f{time.time() - start_time: .2f}s")

In [ ]:
len(geo_coordi_list)

In [ ]:
np_geo_coordi = np.array(geo_coordi_list)
df_geo = df_1.copy() # 깊복.. 깊은복사

df_geo['road_address'] = road_add_list
df_geo['jibun_address'] = jibun_add_list
df_geo['latitude'] = np_geo_coordi[:, 0]
df_geo['longitude'] = np_geo_coordi[:, 1]
df_geo

In [ ]:
# os.getcwd()

In [ ]:
# os.listdir('/content/drive/MyDrive/Colab Notebooks')
# df_geo.to_csv('/content/drive/MyDrive/Colab Notebooks/공모전/천안시/천안시_지오코딩 데이터_추가_0820.csv', encoding = 'utf-8-sig', index = False)

### 2. 법정동 매핑

좌표 위치 + shp polygon 조인

In [ ]:
cheon_shp = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/공모전/천안시/천안 shp/cheon_.shp', encoding='utf-8')
# df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/공모전/천안시/천안시_지오코딩 데이터_추가_0820.csv', encoding='utf-8-sig')
df = df_geo.copy()

In [ ]:
df.head()

In [ ]:
cheon_shp.head()

In [ ]:
cheon_shp.plot(color='gray', edgecolor="w")

In [ ]:
df['geometry'] = gpd.points_from_xy(df.longitude, df.latitude, crs="EPSG:4326")
geo_df = gpd.GeoDataFrame(data=df, geometry='geometry')
geo_df = geo_df.to_crs(epsg=5181)

In [ ]:
geo_df_2 = gpd.sjoin(left_df=geo_df, right_df=cheon_shp, how='left', predicate='within')
# crashes2.to_file(r'/home/bera/Desktop/crashes_with_wards.gpkg')
geo_df_2

In [ ]:
fin_df = pd.DataFrame(geo_df_2)[['지번주소', 'road_address', 'jibun_address', 'latitude', 'longitude', 'ADM_CD', 'ADM_NM']]
fin_df.head()

In [ ]:
fin_df.to_csv('/content/drive/MyDrive/Colab Notebooks/공모전/천안시/행정동 추가_0820.csv', encoding = 'utf-8-sig', index = False)